<a href="https://colab.research.google.com/github/lipeng2021/-python1/blob/main/5_1%E5%8D%B7%E7%A7%AF%E7%A5%9E%E7%BB%8F%E7%BD%91%E7%BB%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torch.nn as nn

In [3]:
def corr2d(x,k):
  h,w = k.shape
  y = torch.zeros(x.shape[0]-h+1,x.shape[1]-w+1)
  for i in range(y.shape[0]):
    for j in range(y.shape[1]):
      y[i,j] = (x[i:i+h,j:j+w]*k).sum()
  return y 

In [4]:
x = torch.arange(9).view(3,3)
k = torch.arange(4).view(2,2)
corr2d(x,k)

tensor([[19., 25.],
        [37., 43.]])

In [5]:
class Cov2D(nn.Module):
  def __init__(self,kernel_size):
    super(Cov2D,self).__init__()
    self.weight = nn.Parameter(torch.randn(kernel_size))
    print(self.weight)
    self.bias = nn.Parameter(torch.randn(1))
    print(self.bias)
  def forward(self,x):
    return corr2d(x,self.weight)+self.bias

In [6]:
x = torch.ones(6,8)
x[:,2:5]=0
print(x)
k = torch.tensor([[1,-1]])
print(k)
y = corr2d(x,k)
print(y)

tensor([[1., 1., 0., 0., 0., 1., 1., 1.],
        [1., 1., 0., 0., 0., 1., 1., 1.],
        [1., 1., 0., 0., 0., 1., 1., 1.],
        [1., 1., 0., 0., 0., 1., 1., 1.],
        [1., 1., 0., 0., 0., 1., 1., 1.],
        [1., 1., 0., 0., 0., 1., 1., 1.]])
tensor([[ 1, -1]])
tensor([[ 0.,  1.,  0.,  0., -1.,  0.,  0.],
        [ 0.,  1.,  0.,  0., -1.,  0.,  0.],
        [ 0.,  1.,  0.,  0., -1.,  0.,  0.],
        [ 0.,  1.,  0.,  0., -1.,  0.,  0.],
        [ 0.,  1.,  0.,  0., -1.,  0.,  0.],
        [ 0.,  1.,  0.,  0., -1.,  0.,  0.]])


In [7]:
cov2d = Cov2D(kernel_size=(1,2))
step = 20
lr = 0.01
for i in range(step):
  y_hat = cov2d(x)
  l = ((y_hat-y)**2).sum()
  l.backward()
  cov2d.weight.data -=lr*cov2d.weight.grad
  cov2d.bias.data -=lr*cov2d.bias.grad
  
  cov2d.weight.grad.fill_(0)
  cov2d.weight.grad.fill_(0)

  if (i+1)%5==0:
    print('epoch:%d,loss:%f'%(i+1,l.item()))
  
print('weight:',cov2d.weight.data)
print('bias:',cov2d.bias.data)



Parameter containing:
tensor([[ 0.3885, -1.5317]], requires_grad=True)
Parameter containing:
tensor([0.1174], requires_grad=True)
epoch:5,loss:2.974430
epoch:10,loss:0.003150
epoch:15,loss:0.000557
epoch:20,loss:0.000148
weight: tensor([[ 0.9969, -0.9969]])
bias: tensor([1.7060e-05])


In [8]:
def com_cov2d(cov2d,x):
  x = x.view((1,1)+x.shape)
  y = conv2d(x)
  return y.view(y.shape[2:])

In [9]:
conv2d = nn.Conv2d(in_channels=1,out_channels=1,kernel_size=3,padding=1)
x = torch.randn(8,8)
com_cov2d(conv2d,x).shape

torch.Size([8, 8])

In [10]:
x = torch.randn(8,8)
print((1,1)+x.shape)

(1, 1, 8, 8)


In [11]:
from torch.nn import common_types
conv2d = nn.Conv2d(in_channels=1,out_channels=1,kernel_size=(5,3),padding=(2,1))
com_cov2d(conv2d,x).shape

torch.Size([8, 8])

In [12]:
conv2d=nn.Conv2d(1,1,3,2,1)
com_cov2d(conv2d,x).shape

torch.Size([4, 4])

In [13]:
x = torch.randn(9,12)
conv2d=nn.Conv2d(1,1,(3,5),(3,4),(0,1))
com_cov2d(conv2d,x).shape

torch.Size([3, 3])

In [22]:
x = torch.arange(16,dtype=torch.float).view(1,1,4,4)
print(x)
pool2d = nn.MaxPool2d(3,stride=3)
pool2d(x)

tensor([[[[ 0.,  1.,  2.,  3.],
          [ 4.,  5.,  6.,  7.],
          [ 8.,  9., 10., 11.],
          [12., 13., 14., 15.]]]])


tensor([[[[10.]]]])